In [1]:
# BIBLIOTECAS
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from openpyxl import Workbook

In [2]:
df_coord = pd.read_excel("C:/Users/Nicolas Lessa/TRABALHOS EM PYTHON/Trabalho Temp/Coordenadas cidades populosas.xlsx") #FIXO PARA TODAS AS ANÁLISES

def analisar_arquivo(arquivo):
    # Carregar o arquivo CSV
    df = pd.read_csv(arquivo)

    # Aplicar seu modelo de análise aqui
    
    #CRIAÇÃO DOS PONTOS PARA COLOCAR NO MAPA
    geometry = [Point(xy) for xy in zip(df['long'], df['lat'])]
    gdf = GeoDataFrame(df, geometry=geometry)   

    #FUNÇÃO QUE PARA REALIZAR O PLOT DOS QUADRADOS NO MAPA
    cidadeX = []
    cidadeY = []

    for i in range(len(df_coord)):
    #SEPARANDO OS Xs e Ys DAS COORDENADAS
        x1 = df_coord["COORD 1"][i].split(",")[1]
        y1 = df_coord["COORD 1"][i].split(",")[0]
        x2 = df_coord["COORD 2"][i].split(",")[1]
        y2 = df_coord["COORD 2"][i].split(",")[0]
        x3 = df_coord["COORD 3"][i].split(",")[1]
        y3 = df_coord["COORD 3"][i].split(",")[0]
        x4 = df_coord["COORD 4"][i].split(",")[1]
        y4 = df_coord["COORD 4"][i].split(",")[0]
        cidadeX.append([x1,x2,x3,x4])
        cidadeY.append([y1,y2,y3,y4])

    #CRIANDO COLUNAS NO df_coord PARA CADA COMPONENTE DAS COORDENADAS DOS DELIMITANTES DAS CIDADES A SEREM ANALISADAS
    #cidadeX/Y[ÍNDICE][COORDENADA 1,2,3,4 (0,1,2,3)]
    #TRANSFORMANDO OS ELEMENTOS DAS LISTAS DE STR PARA FLOAT
    for i in range(len(cidadeX)):
        for k in range(len(cidadeX[0])):
            cidadeX[i][k] = float(cidadeX[i][k])
    for i in range(len(cidadeY)):
        for k in range(len(cidadeY[0])):
            cidadeY[i][k] = float(cidadeY[i][k])

    #CRIANDO COLUNAS VAZIAS PARA COLOCAR CADA COORDENADA DOS PONTOS
    df_coord["1X"] = pd.Series(0, index=df.index, dtype=int)
    df_coord["2X"] = pd.Series(0, index=df.index, dtype=int)
    df_coord["3X"] = pd.Series(0, index=df.index, dtype=int)
    df_coord["4X"] = pd.Series(0, index=df.index, dtype=int)

    df_coord["1Y"] = pd.Series(0, index=df.index, dtype=int)
    df_coord["2Y"] = pd.Series(0, index=df.index, dtype=int)
    df_coord["3Y"] = pd.Series(0, index=df.index, dtype=int)
    df_coord["4Y"] = pd.Series(0, index=df.index, dtype=int)
    #SUBSTITUINDO OS VALORES DE CADA COORDENADA
    for i in range(len(df_coord)):
        df_coord.loc[i,"1X"] = cidadeX[i][0]
        df_coord.loc[i,"2X"] = cidadeX[i][1]
        df_coord.loc[i,"3X"] = cidadeX[i][2]
        df_coord.loc[i,"4X"] = cidadeX[i][3]

        df_coord.loc[i,"1Y"] = cidadeY[i][0]
        df_coord.loc[i,"2Y"] = cidadeY[i][1]
        df_coord.loc[i,"3Y"] = cidadeY[i][2]
        df_coord.loc[i,"4Y"] = cidadeY[i][3]
    
    #TRACEJANDO LINHAS DO CONTORNO PARA FECHAR UM POLÍGONO, ASSIM PODENDO COMPARAR OS PONTOS QUE ESTÃO DENTRO DO POLÍGONO
    #COLUNA NOVA
    df_coord["poligonos"] = pd.Series(0, index=df.index, dtype=int)
    #SUBSTITUINDO OS VALORES NA NOVA COLUNA
    for i in range(len(df_coord)):
        coordenadas = [(df_coord["1X"][i], df_coord["1Y"][i]), (df_coord["2X"][i], df_coord["2Y"][i]), (df_coord["3X"][i], df_coord["3Y"][i]), (df_coord["4X"][i], df_coord["4Y"][i]), (df_coord["1X"][i], df_coord["1Y"][i])]
        linha = gpd.GeoSeries([LineString(coordenadas)])
        linha.polygonize()
        df_coord.loc[i,"poligonos"] = linha.polygonize().geometry.values

    #CHECAGEM DE PONTO A PONTO SE ELE ESTA DENTRO OU FORA DAS CIDADES A SEREM ANALISADAS
    temp_cidade = pd.DataFrame(columns=["temperatura","cidade"])
    for i in range(len(df_coord)):
        for k in range(len(gdf)):
            if gdf["geometry"][k].within(df_coord["poligonos"][i]) == True: #CHECAR SE OS PONTOS DA gdf ESTÃO DENTRO DOS POLIGONOS
                temp_cidade.loc[k,"temperatura"] = gdf["value"][k] #SE ESTIVER COLETE A TEMPERATURA
                temp_cidade.loc[k,"cidade"] = df_coord["MUNICÍPIO"][i] #EM QUAL CIDADE ESTÃO SENDO ATRIBUÍDOS ESSES VALORES
                temp_cidade.loc[k,"lat"] = gdf["lat"][k] #LATITUDE E
                temp_cidade.loc[k,"long"] = gdf["long"][k] #LONGITUDE DOS PONTOS ANALISADOS

    temp_cidade_media = temp_cidade.groupby("cidade").agg(media=("temperatura","mean")) #AGRUPANDO POR CIDADE E TIRANDO A MÉDIA
    temp_cidade_max = temp_cidade.groupby("cidade").agg(max=("temperatura","max")) #MÁXIMA
    temp_cidade_min = temp_cidade.groupby("cidade").agg(min=("temperatura","min")) #MÍNIMA

    return temp_cidade_media, temp_cidade_max, temp_cidade_min

In [ ]:
diretorio = "C:/Users/Nicolas Lessa/TRABALHOS EM PYTHON/Trabalho Temp/SAMeT_CPTEC_T_2000 - Copia" #PASTA DEZIPADA COM OS ARQUIVOS A SEREM LIDOS
arquivos = [arquivo for arquivo in os.listdir(diretorio) if arquivo.endswith('.csv')]

resultados = pd.DataFrame(columns=['Arquivo', 'temp_media','temp_max','temp_min'])

for i, arquivo in enumerate(arquivos):
    
    resultado = analisar_arquivo(os.path.join(diretorio, arquivo))
    
    temp_cidade_media, temp_cidade_max, temp_cidade_min = resultado

    resultados.loc[i,"Arquivo"] = arquivo
    resultados.loc[i,"temp_cidade_media"] = str(temp_cidade_media)
    resultados.loc[i,"temp_cidade_max"] = str(temp_cidade_max)
    resultados.loc[i,"temp_cidade_min"] = str(temp_cidade_min)

workbook = Workbook()
sheet = workbook.active
sheet.title = "Resultados"
for index, row in resultados.iterrows():
    sheet.append(list(row.values))
workbook.save("resultados_consolidados.xlsx")